In [12]:
import snap
import subprocess

In [17]:
subprocess.check_output(["go", "run", "bfs.go", "-out", "-levels", "5", "252001"])

'Starting from 252001 and searcher for 5 levels (follow out: true, follow in: false)\nFound 6591 nodes, 11443 edges and output edge list to 252001.txt'

In [14]:
G = snap.LoadEdgeList(snap.PNGraph, "252001.txt", 0, 1)

In [15]:
G.GetEdges()

11443

In [16]:
G.GetNodes()

6591